In [6]:
import tensorflow as tf
import numpy as np
import seaborn
import matplotlib.pyplot as plt
% pylab inline

Populating the interactive namespace from numpy and matplotlib


### RANK NET

In [139]:
#### -------------> Import data:

def readDataset(path):
    '''
    Dataset - LETOR 4.0
    Dataset format: svmlight / libsvm format
    <label> <feature-id>:<feature-value>... #docid = <feature-value> inc = <feature-value> prob = <feature-value>
    We have a total of 46 features
    '''

    X_train = [] #<feature-value>[46]
    y_train = [] #<label>
    Query = []   #<query-id><document-id><inc><prob>

    print('Reading training data from file...')

    with open(path, 'r') as file:
        for line in file:
            split = line.split()
            y_train.append(int(split[0]))
            X_train.append(extractFeatures(split))
            Query.append(extractQueryData(split))
    print('Read %d lines from file...' %(len(X_train)))
    return (X_train, y_train, Query)

def extractFeatures(split):
    '''
    Extract the query to document features used
    as input to the neural network
    '''
    features = []
    for i in xrange(2, 138):
        features.append(float(split[i].split(':')[1]))
    return features

def extractQueryData(split):
    '''
    Extract the query features from a dataset line
    Format:
    <query-id><document-id><inc><prob>
    '''
    queryFeatures = [split[1].split(':')[1]]

    return queryFeatures

def extractPairsOfRatedSites(y_train, Query):
    '''
    For each queryid, extract all pairs of documents
    with different relevance judgement and save them in
    a list with the most relevant in position 0
    '''
    pairs = []
    for i in xrange(0, len(Query)):
        for j in xrange(i+1, len(Query)):
            #Only look at queries with the same id
            if(Query[i] != Query[j]):
                break
            #Document pairs found with different rating
            if(Query[i] == Query[j] and y_train[i] != y_train[j]):
                #Sort by saving the largest index in position 0
                if(y_train[i] > y_train[j]):
                    pairs.append([i, j])
                else:
                    pairs.append([j, i])
    print('Found %d document pairs' %(len(pairs)))
    return pairs

def separate_training(X,pairs):
    data1=[]
    data2=[]
    for i in pairs:
        data1.append(X[i[0]])
        data2.append(X[i[1]])
    return np.array(data1),np.array(data2)





In [242]:
#Read training data
X_train, y_train, Query = readDataset('./MSLR-WEB10K/Fold1/train.txt')

# Taking first 500 queries:
len_train=5000
X_train1,y_train1=X_train[0:len_train],y_train[0:len_train]
Query1=Query[0:len_train]

# Extract document pairs
#pairs1 = extractPairsOfRatedSites(y_train1, Query1)
pairs1 = extractPairsOfRatedSites(y_train1, Query1)

X_array=np.array(X_train1)
data1,data2=separate_training(X_array,pairs1)

Reading training data from file...
Read 723412 lines from file...
Found 213868 document pairs


In [243]:
# Constant, variables and place holders:
nDim=data1.shape[1]
N=data1.shape[0]
first_layer=10
output_layer=1
A = tf.placeholder(tf.float32, [None, nDim])
B = tf.placeholder(tf.float32, [None, nDim])

In [244]:
P_AB = tf.placeholder(tf.float32, [None, 1])
P_true = tf.placeholder(tf.float32, [None, 1])

In [245]:
# Weights for the first layer to hidden layer:
weights1 = tf.Variable(tf.random_normal([nDim, first_layer]))
biases1 = tf.Variable(tf.random_normal([first_layer]))

In [246]:
# Hidden Layer nodes:
hiddenA = tf.matmul(A, weights1) + biases1
hiddenB = tf.matmul(B, weights1) + biases1

In [247]:
# Activations hidden layer
act_hiddenA = tf.nn.sigmoid(hiddenA)
act_hiddenB = tf.nn.sigmoid(hiddenB)

In [248]:
# Weights from hidden layer to output layer:
weights2 = tf.Variable(tf.random_normal([first_layer,output_layer]))
biases2 = tf.Variable(tf.random_normal([output_layer]))

In [249]:
# Output layer:
outputA = tf.matmul(act_hiddenA, weights2) + biases2
outputB = tf.matmul(act_hiddenB, weights2) + biases2

Oi = tf.nn.sigmoid(outputA)
Oj = tf.nn.sigmoid(outputB)
Oij=Oi-Oj

In [250]:
# Probability:
Pij=tf.exp(Oij)/(1+tf.exp(Oij))

In [251]:
# Cross entropy and cost:
cross_entropy = -tf.reduce_sum(tf.log(Pij))
#cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Pij,labels=P_true)
#cost = tf.reduce_mean(cross_entropy)

In [252]:
# Optimizer:
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(cost)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.00015).minimize(cross_entropy)

In [253]:
# Start session:
batch_size = 100
session = tf.Session()
session.run(tf.global_variables_initializer())

In [254]:
P_target=np.ones([data1.shape[0],1])

In [257]:
def optimize(num_iterations):
    for i in range(num_iterations):
        # Get a batch of training examples.
        # x_batch now holds a batch of images and
        # y_true_batch are the true labels for those images.
#        A_batch, y_true_batch = data.train.next_batch(batch_size)
        indices = np.random.choice(data1.shape[0], batch_size)
        A_batch, B_batch, target_batch = data1[indices], data2[indices] , P_target[indices]
        
        # Put the batch into a dict with the proper names
        # for placeholder variables in the TensorFlow graph.
        # Note that the placeholder for y_true_cls is not set
        # because it is not used during training.
        #
        #feed_dict_train = {A: data1,B:data2, P_true: target} --> working
        feed_dict_train = {A: A_batch,B:B_batch, P_true: target_batch} 
        # Run the optimizer using this batch of training data.
        # TensorFlow assigns the variables in feed_dict_train
        # to the placeholder variables and then runs the optimizer.
        session.run(optimizer, feed_dict=feed_dict_train)
        #oi = session.run(Oi, feed_dict= feed_dict_train)
        #oj = session.run(Oj, feed_dict= feed_dict_train)
        #oij = session.run(Oij, feed_dict= feed_dict_train)
        #pij = session.run(Pij, feed_dict= feed_dict_train)
        c_e = session.run(cross_entropy, feed_dict= feed_dict_train)
        #error = session.run(cost, feed_dict= feed_dict_train)
        print('Epoch', i, "pij ",c_e)
        


In [258]:
optimize(num_iterations=100)

('Epoch', 0, 'pij ', 71.417557)
('Epoch', 1, 'pij ', 68.537781)
('Epoch', 2, 'pij ', 71.048203)
('Epoch', 3, 'pij ', 70.87294)
('Epoch', 4, 'pij ', 71.197083)
('Epoch', 5, 'pij ', 71.768738)
('Epoch', 6, 'pij ', 67.665123)
('Epoch', 7, 'pij ', 71.295334)
('Epoch', 8, 'pij ', 70.438354)
('Epoch', 9, 'pij ', 70.881599)
('Epoch', 10, 'pij ', 68.809731)
('Epoch', 11, 'pij ', 69.518784)
('Epoch', 12, 'pij ', 69.393433)
('Epoch', 13, 'pij ', 70.630615)
('Epoch', 14, 'pij ', 68.610779)
('Epoch', 15, 'pij ', 68.944077)
('Epoch', 16, 'pij ', 69.500565)
('Epoch', 17, 'pij ', 69.251251)
('Epoch', 18, 'pij ', 68.300026)
('Epoch', 19, 'pij ', 70.596436)
('Epoch', 20, 'pij ', 72.004471)
('Epoch', 21, 'pij ', 71.970871)
('Epoch', 22, 'pij ', 70.522217)
('Epoch', 23, 'pij ', 68.116547)
('Epoch', 24, 'pij ', 71.126862)
('Epoch', 25, 'pij ', 71.565155)
('Epoch', 26, 'pij ', 68.773254)
('Epoch', 27, 'pij ', 69.646286)
('Epoch', 28, 'pij ', 70.025238)
('Epoch', 29, 'pij ', 70.252258)
('Epoch', 30, 'pij ',

In [37]:
prediction = tf.round(Pij * 2) / 2
prediction= tf.cast(prediction, tf.float64)
correct_prediction = tf.equal(P_target, prediction)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### Validation:

In [38]:
#Read training data
X_val, y_val, Query_val = readDataset('./MSLR-WEB10K/Fold1/vali.txt')

# Extract document pairs
pairs_val = extractPairsOfRatedSites(y_val, Query_val)
X_val_array=np.array(X_val)
data1_val,data2_val=separate_training(X_val_array,pairs_val)

Reading training data from file...
Read 235259 lines from file...
Found 10103042 document pairs


In [44]:
P_target_val=np.ones([data1_val.shape[0],1])

In [45]:
len_test=500

In [51]:
feed_dict_test = {A: data1_val[0:len_test],B:data2_val[0:len_test] ,P_true: P_target_val[0:len_test]} 

In [52]:
def print_accuracy():
    # Use TensorFlow to compute the accuracy.
    acc = session.run(accuracy, feed_dict=feed_dict_test)
    
    # Print the accuracy.
    print("Accuracy on test-set: {0:.1%}".format(acc))

In [53]:
print_accuracy()

InvalidArgumentError: Incompatible shapes: [213868,1] vs. [500,1]
	 [[Node: Equal_3 = Equal[T=DT_DOUBLE, _device="/job:localhost/replica:0/task:0/cpu:0"](Equal_3/x, Cast_4)]]

Caused by op u'Equal_3', defined at:
  File "/Users/Chelo/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/Chelo/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/Chelo/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/Chelo/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/Users/Chelo/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/Users/Chelo/anaconda2/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 162, in start
    super(ZMQIOLoop, self).start()
